Computes Vanilla CSM model hadron yield ratios that are not normalized to the high-multiplicity limit. Also outputs (dNch/dy) and (dNch/dEta), under the assumption (dNch/dEta) = (dNch/dy) / 1.2 (see https://arxiv.org/abs/1908.02792)

In [1]:
#include "inc/loadFISTlibs.h"

###############################################################################
#                                                                             #
# This is Thermal-FIST version 1.2.2                                          #
#                                                                             #
# Copyright (c) 2019 Volodymyr Vovchenko <vovchenko@fias.uni-frankfurt.de>    #
#                                                                             #
# Distributed under the GNU General Public License 3.0 (GPLv3 or later)       #
#                                                                             #
# Please cite when using this code:                                           #
# V. Vovchenko, H. Stoecker, arXiv:1901.05249 [nucl-th]                       #
#                                                                             #
# The latest version is available at https://github.com/vlvovch/Thermal-FIST  #
#                                       

In [2]:
using namespace std;

In [3]:
using namespace thermalfist;

In [4]:
bool useWidth  = true;
bool useQStats = true;

double Tch = 0.155;

In [5]:
// Particle list
//ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list-withnuclei.dat");
ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list.dat");

Fill the list of the considered hadron yield ratios

In [6]:
vector<int> pdgs1, pdgs2;
vector<string> names1, names2;

names1.push_back("K");
names2.push_back("pi");
pdgs1.push_back(321);
pdgs2.push_back(211);

names1.push_back("Xi");
names2.push_back("pi");
pdgs1.push_back(3312);
pdgs2.push_back(211);

names1.push_back("phi");
names2.push_back("pi");
pdgs1.push_back(333);
pdgs2.push_back(211);

names1.push_back("p");
names2.push_back("pi");
pdgs1.push_back(2212);
pdgs2.push_back(211);

names1.push_back("Om");
names2.push_back("pi");
pdgs1.push_back(3334);
pdgs2.push_back(211);

names1.push_back("La");
names2.push_back("pi");
pdgs1.push_back(3122);
pdgs2.push_back(211);

names1.push_back("K*");
names2.push_back("pi");
pdgs1.push_back(323);
pdgs2.push_back(211);

Compute the limiting GCE yield ratios values

In [7]:
vector<double> ratiosGCE;
{
    ThermalModelIdeal modelgce(&parts);

    if (useWidth)
        modelgce.SetUseWidth(ThermalParticle::eBW);
    else
        modelgce.SetUseWidth(ThermalParticle::ZeroWidth);
    modelgce.SetStatistics(useQStats);
    // Include quantum statistics for mesons only
    if (useQStats){
        for (int i = 0; i < modelgce.TPS()->Particles().size(); ++i) {
            ThermalParticle &part = modelgce.TPS()->Particle(i);
            if (part.BaryonCharge() != 0)
                part.UseStatistics(false);
        }
    }

    modelgce.SetTemperature(Tch);
    modelgce.SetBaryonChemicalPotential(0.);
    modelgce.SetElectricChemicalPotential(0.);
    modelgce.SetStrangenessChemicalPotential(0.);

    modelgce.FillChemicalPotentials();

    modelgce.CalculateDensities();

    for (int i = 0; i < names1.size(); ++i) {
        ratiosGCE.push_back(modelgce.GetDensity(pdgs1[i], 1) / modelgce.GetDensity(pdgs2[i], 1));
    }
}

Prepare the output (on the screen and to a file)

In [8]:
vector<std::ostream*> outs;
outs.push_back(&std::cout);

outs.push_back(NULL);

Routine performing the (logarithmic scale) scan over Vc and dNpi/dy.

Parameter k determines Vc = k dV/dy

In [9]:
void PerformScan(ThermalModelBase *model, 
                   double k,
                   const string& filename)
{
    ofstream fout(filename);
    outs[1] = &fout;
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "dNpipm/dy";
        *out << setw(15) << "Vc[fm3]";
        *out << setw(15) << "dNch/dy";
        *out << setw(15) << "dNch/deta";

        for (int i = 0; i < names1.size(); ++i) {
            *out << setw(15) << "(" + (names1[i] + "/" + names2[i]) + ")toHML";
        }

        *out << endl;
    }
    
    double Vmin = 1. * k;
    double Vmax = 15000. * k;
    int iters = 30;
    double logVmin = log10(Vmin), logVmax = log10(Vmax);
    double dlogV = (logVmax - logVmin) / (iters - 1);
    
    double dNchdyOverdNchddeta = 1.20; // pp value from https://arxiv.org/abs/1908.02792
    
    for (double logV = logVmin; logV <= logVmax + 1.e-3; logV += dlogV) {
        double V = pow(10., logV);

        model->SetVolume(V);
        model->SetCanonicalVolume(V);
        model->CalculateDensities();

        for(std::ostream* out : outs) {
            *out << setw(15) << 2. * model->GetDensity(211, 1) * V / k; // Charged pions dN/dy
            *out << setw(15) << V; // Vc
            *out << setw(15) << model->ChargedMultiplicity(0) / k;
            *out << setw(15) << model->ChargedMultiplicity(0) / k / dNchdyOverdNchddeta;

            for (int i = 0; i < names1.size(); ++i) {
                *out << setw(15) << model->GetDensity(pdgs1[i], 1) / model->GetDensity(pdgs2[i], 1);
            }

            *out << endl;
        }
    }
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";

        for (int i = 0; i < ratiosGCE.size(); ++i) {
            *out << setw(15) << ratiosGCE[i];
        }

        *out << endl;
    }
    
    fout.close();
}

Perform the vanilla CSM scans

In [10]:
ThermalModelCanonical modelce(&parts);

if (useWidth)
    modelce.SetUseWidth(ThermalParticle::eBW);
else
    modelce.SetUseWidth(ThermalParticle::ZeroWidth);

modelce.SetStatistics(useQStats);
// Include quantum statistics for mesons only
if (useQStats){
    for (int i = 0; i < modelce.TPS()->Particles().size(); ++i) {
        ThermalParticle &part = modelce.TPS()->Particle(i);
        if (part.BaryonCharge() != 0)
            part.UseStatistics(false);
    }
}

modelce.SetTemperature(Tch);
modelce.SetBaryonChemicalPotential(0.);
modelce.SetElectricChemicalPotential(0.);
modelce.SetStrangenessChemicalPotential(0.);

modelce.SetBaryonCharge(0);
modelce.SetElectricCharge(0);
modelce.SetStrangeness(0);

modelce.FillChemicalPotentials();

In [11]:
PerformScan(&modelce, 1., "out/VanillaCSM.Extras.Vc.eq.dVdy.dat");

      dNpipm/dy        Vc[fm3]        dNch/dy      dNch/deta    (K/pi)toHML   (Xi/pi)toHML  (phi/pi)toHML    (p/pi)toHML   (Om/pi)toHML   (La/pi)toHML   (K*/pi)toHML
      0.0768496              1      0.0077515     0.00645958      0.0617676     1.5189e-05      0.0743313    0.000861875    3.96148e-07    0.000333069     0.00682599
       0.111546        1.39317      0.0151233      0.0126027      0.0636195    2.18256e-05      0.0713456     0.00119742    5.87085e-07    0.000468215     0.00795567
       0.164161        1.94091      0.0295411      0.0246176      0.0662119    3.17536e-05      0.0675398     0.00166367    8.91101e-07    0.000661047     0.00947104
       0.245806        2.70402      0.0577536       0.048128      0.0698437    4.69978e-05      0.0628423     0.00231337    1.39685e-06    0.000939331      0.0114818
       0.375664        3.76715       0.112861      0.0940505      0.0749287    7.11951e-05      0.0572878     0.00322419    2.28386e-06     0.00134736      0.0141161
    

In [12]:
PerformScan(&modelce, 3., "out/VanillaCSM.Extras.Vc.eq.3dVdy.dat");

      dNpipm/dy        Vc[fm3]        dNch/dy      dNch/deta    (K/pi)toHML   (Xi/pi)toHML  (phi/pi)toHML    (p/pi)toHML   (Om/pi)toHML   (La/pi)toHML   (K*/pi)toHML
      0.0933613              3      0.0237505      0.0197921      0.0712572    5.33777e-05      0.0611887     0.00256605    1.62127e-06     0.00105048      0.0122335
       0.143668         4.1795      0.0463811      0.0386509      0.0769044    8.15653e-05      0.0553986     0.00358046    2.69182e-06     0.00151233       0.015092
       0.225991        5.82274      0.0900831      0.0750692      0.0847562     0.00012872      0.0490675       0.005023    4.72354e-06     0.00220831      0.0187609
       0.361925        8.11205       0.172489       0.143741      0.0954818    0.000210848      0.0426882     0.00710397    8.81448e-06     0.00327962      0.0233675
       0.583287        11.3014       0.320731       0.267276       0.109534    0.000357893      0.0369064      0.0101361    1.74335e-05     0.00494624       0.028937
    

In [13]:
PerformScan(&modelce, 6., "out/VanillaCSM.Extras.Vc.eq.6dVdy.dat");

      dNpipm/dy        Vc[fm3]        dNch/dy      dNch/deta    (K/pi)toHML   (Xi/pi)toHML  (phi/pi)toHML    (p/pi)toHML   (Om/pi)toHML   (La/pi)toHML   (K*/pi)toHML
       0.117839              6      0.0478035      0.0398362      0.0855983    0.000134384      0.0484836     0.00518106    4.98489e-06     0.00228708      0.0191375
       0.188941          8.359      0.0913563      0.0761303      0.0966135    0.000220884      0.0421307     0.00733355    9.35435e-06     0.00340191      0.0238328
       0.304414        11.6455       0.169261       0.141051       0.110965    0.000375949       0.036435      0.0104706    1.85861e-05     0.00513596      0.0294824
       0.483579        16.2241       0.298277       0.248564       0.128029    0.000651421      0.0319589      0.0149846    3.82521e-05      0.0077733      0.0357686
       0.745693        22.6029       0.494359       0.411966       0.145291     0.00110873      0.0288787       0.021123    7.80773e-05      0.0114913      0.0418225
    